In [1]:
# read the text file "OK.population" and store in the list called population
population = list()

# open the text file for reading
filepath = "C:\\districting-data\\"
filename = "OK.population"
file = open( filepath + filename ,"r")

# while the current line is not empty, read in a new county population
line = file.readline()

while line != "":
    # split the line into two "words": 
    #    word[0]: the county's number
    #    word[1]: the county's population
    words = line.split() 
    county_number = words[0]
    county_population = int(words[1]) # cast the string as type int
    
    # append to population list
    population.append(county_population)
    
    # read next line
    line = file.readline() 

file.close()
print("population = ",population)

population =  [255755, 11943, 45837, 20081, 50976, 15034, 124098, 6193, 27469, 60580, 11154, 11629, 69967, 14003, 42391, 12769, 41848, 47472, 46987, 73085, 27576, 34273, 52431, 115541, 40069, 8878, 47557, 10957, 16577, 86905, 69442, 33151, 77350, 29600, 41487, 15840, 6239, 5925, 25482, 13488, 6472, 7992, 20252, 15205, 3685, 4527, 4810, 718633, 50384, 10536, 20640, 12191, 37492, 3647, 9446, 603403, 22119, 14182, 22683, 11561, 2922, 7527, 5642, 15029, 9423, 5636, 11572, 45048, 41259, 42416, 46562, 70990, 26446, 2475, 4151, 31848, 34506]


In [2]:
# we are to solve the following task:
# input: a population vector, desired number of districts k
# output: a partition of the populations into k districts (not necessarily connected!) 
#            to minimize the difference between most and least populated districts

import gurobipy as gp
from gurobipy import GRB

In [3]:
n = len(population) # number of counties
k = 5               # desired number of districts

# create model 
m = gp.Model()

# create variables
x = m.addVars(n,k,vtype=GRB.BINARY) # x_ij equals one when county i is assigned to district j
y = m.addVar()                      # the population of least-populated district
z = m.addVar()                      # the population of most-populated district

Academic license - for non-commercial use only - expires 2021-04-22
Using license file C:\Users\buchanan\gurobi.lic


In [4]:
# objective is to minimize absolute population deviation
m.setObjective(z-y,GRB.MINIMIZE)

# add constraints saying that each county i is assigned to one district
m.addConstrs( sum(x[i,j] for j in range(k)) == 1 for i in range(n) )

# add constraints saying that each district has population at least y
m.addConstrs( y <= sum( population[i] * x[i,j] for i in range(n) ) for j in range(k) )

# add constraints saying that each district has population at most z
m.addConstrs( sum( population[i] * x[i,j] for i in range(n)) <= z for j in range(k) )

{0: <gurobi.Constr *Awaiting Model Update*>,
 1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>,
 3: <gurobi.Constr *Awaiting Model Update*>,
 4: <gurobi.Constr *Awaiting Model Update*>}

In [5]:
# solve IP model
m.optimize()

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 87 rows, 387 columns and 1165 nonzeros
Model fingerprint: 0x235259e8
Variable types: 2 continuous, 385 integer (385 binary)
Coefficient statistics:
  Matrix range     [1e+00, 7e+05]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 1149841.0000
Presolve time: 0.00s
Presolved: 87 rows, 387 columns, 1165 nonzeros
Variable types: 0 continuous, 387 integer (385 binary)

Root relaxation: objective 0.000000e+00, 125 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.00000    0    7 1149841.00    0.00000   100%     -    0s
H    0     0                    85663.000000    0.00000   100%     -    0s
H    0     0      

In [7]:
# print the absolute population deviation
print("The absolute population deviation is",m.objval,"person(s).")

# retrieve the districts and their populations
districts = [ [i for i in range(n) if x[i,j].x > 0.5] for j in range(k)]
district_populations = [ sum(population[i] for i in district) for district in districts ]

# print district info
for j in range(k):
    print("District",j,"has population",district_populations[j],"and contains counties",districts[j])

The absolute population deviation is 1.0 person(s).
District 0 has population 750270 and contains counties [0, 1, 5, 9, 10, 11, 12, 17, 26, 30, 33, 34, 35, 37, 42, 45, 58, 64]
District 1 has population 750270 and contains counties [39, 40, 41, 44, 47]
District 2 has population 750270 and contains counties [22, 46, 55, 60, 65, 70, 76]
District 3 has population 750270 and contains counties [2, 3, 6, 7, 8, 13, 14, 15, 16, 18, 19, 20, 25, 27, 31, 36, 43, 49, 50, 53, 54, 61, 62, 63, 66, 71, 73, 74, 75]
District 4 has population 750271 and contains counties [4, 21, 23, 24, 28, 29, 32, 38, 48, 51, 52, 56, 57, 59, 67, 68, 69, 72]
